# О проекте


Задание относится к проекту UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 


## Цель проекта

Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

## Средства достижения целей

Реализовать достижение целей можно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы.

## Задача проекта

Чтобы определиться с параметрами будущей модели, нужно провести разведывательный анализ данных и составьте отчёт по его результатам.

### Импорт библиотек

In [1]:
import warnings; warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import os
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from itertools import combinations

from scipy.stats import ttest_ind
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats import weightstats 


%matplotlib inline

### Инструменты для предобработки

### Загрузка первичных данных

In [2]:
# 
incoming_files = 'stud_math.csv'

# 
if incoming_files not in os.listdir():
    print('\n!!! ВНИМАНИЕ !!!'*5)
    print('Не найден входной файл данных:', incoming_files)
else:
    print(r'[OK] Успешно пройдена проверка наличия входного файла датасета.')


[OK] Успешно пройдена проверка наличия входного файла датасета.


In [3]:
# 
stud = pd.read_csv(incoming_files)

# 
stud.shape

(395, 30)

In [4]:
# показывать больше строк
pd.set_option('display.max_rows', 50)

# показывать больше колонок
pd.set_option('display.max_columns', stud.shape[1] + 5) 

In [5]:
stud.sample(9)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
80,GP,M,15,U,GT3,T,2.0,3.0,other,services,course,father,1.0,1.0,0.0,yes,yes,yes,yes,no,-3.0,yes,NaN,yes,3.0,2.0,2.0,3.0,2.0,60.0
130,GP,F,15,R,GT3,T,3.0,4.0,services,teacher,course,father,2.0,3.0,2.0,no,yes,no,no,yes,-9.0,yes,yes,yes,4.0,2.0,2.0,5.0,0.0,0.0
285,GP,M,17,U,GT3,T,1.0,1.0,other,other,reputation,father,1.0,2.0,0.0,no,no,yes,no,no,-6.0,yes,yes,no,4.0,3.0,3.0,4.0,2.0,55.0
119,GP,M,15,U,GT3,T,3.0,4.0,other,other,reputation,father,1.0,1.0,0.0,no,no,no,no,yes,-3.0,yes,yes,no,3.0,4.0,3.0,4.0,6.0,65.0
354,MS,M,17,NaN,GT3,T,4.0,3.0,services,other,home,mother,2.0,2.0,0.0,no,yes,yes,yes,no,-6.0,yes,yes,yes,4.0,5.0,5.0,2.0,4.0,55.0
310,GP,F,19,U,LE3,T,1.0,NaN,services,services,home,other,NaN,2.0,1.0,no,no,no,yes,no,-6.0,yes,no,yes,4.0,2.0,4.0,3.0,0.0,0.0
348,GP,F,17,NaN,GT3,T,4.0,3.0,health,other,reputation,mother,NaN,3.0,0.0,no,yes,yes,yes,yes,-9.0,yes,yes,yes,NaN,4.0,3.0,4.0,0.0,75.0
291,GP,F,17,U,GT3,T,4.0,3.0,NaN,services,reputation,NaN,1.0,3.0,0.0,no,yes,yes,no,yes,-9.0,yes,yes,no,4.0,NaN,2.0,3.0,0.0,75.0
36,GP,M,15,U,NaN,T,4.0,3.0,teacher,services,home,mother,1.0,3.0,0.0,no,yes,no,yes,yes,-9.0,yes,yes,no,5.0,4.0,3.0,4.0,2.0,90.0


In [6]:
stud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

### Первичная обработка данных

In [7]:
# Удаляем столбец не заявленный в описании

stud.drop(['studytime, granular'], inplace=True, axis = 1)

# приводим все названия колонок к нижнему регистру
stud.columns = [str.lower(x) for x in stud.columns]


In [8]:
stud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      395 non-null    object 
 1   sex         395 non-null    object 
 2   age         395 non-null    int64  
 3   address     378 non-null    object 
 4   famsize     368 non-null    object 
 5   pstatus     350 non-null    object 
 6   medu        392 non-null    float64
 7   fedu        371 non-null    float64
 8   mjob        376 non-null    object 
 9   fjob        359 non-null    object 
 10  reason      378 non-null    object 
 11  guardian    364 non-null    object 
 12  traveltime  367 non-null    float64
 13  studytime   388 non-null    float64
 14  failures    373 non-null    float64
 15  schoolsup   386 non-null    object 
 16  famsup      356 non-null    object 
 17  paid        355 non-null    object 
 18  activities  381 non-null    object 
 19  nursery     379 non-null    o

In [9]:
# dtypes: float64(11), int64(1), object(17)

dig_columns = ['age','medu', 'fedu', 'traveltime', 'studytime', 'failures',
              'famrel', 'freetime', 'goout', 'health', 'absences', 'score']

stud.loc[:, dig_columns].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         395 non-null    int64  
 1   medu        392 non-null    float64
 2   fedu        371 non-null    float64
 3   traveltime  367 non-null    float64
 4   studytime   388 non-null    float64
 5   failures    373 non-null    float64
 6   famrel      368 non-null    float64
 7   freetime    384 non-null    float64
 8   goout       387 non-null    float64
 9   health      380 non-null    float64
 10  absences    383 non-null    float64
 11  score       389 non-null    float64
dtypes: float64(11), int64(1)
memory usage: 37.2 KB


In [10]:
all_columns = stud.columns
all_columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'pstatus', 'medu', 'fedu',
       'mjob', 'fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout',
       'health', 'absences', 'score'],
      dtype='object')

In [11]:
other_columns = [x for x in all_columns if x not in dig_columns]

other_columns


['school',
 'sex',
 'address',
 'famsize',
 'pstatus',
 'mjob',
 'fjob',
 'reason',
 'guardian',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [59]:
# 
def column_content_analysis(column_name, column_type='str', more_than = 10):
    print('-'*60)
    print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
    print('-'*60)
    print('|  Всего значений:', stud[column_name].count())
    print('|  Количество уникальных значений:', stud[column_name].nunique())
    print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
          (stud[column_name].value_counts()>more_than).sum())
    print('|  Уникальные значения:', stud[column_name].unique())
#     return



In [60]:
column_content_analysis('address')

------------------------------------------------------------
|	 Отчет по колонке [ address ] 					
------------------------------------------------------------
|  Всего значений: 378
|  Количество уникальных значений: 2
|  Значений, встретившихся в столбце более 10 раз: 2
|  Уникальные значения: ['U' nan 'R']


In [14]:
stud.address.unique()

array(['U', nan, 'R'], dtype=object)

In [15]:
# stud.address = stud.address.apply(lambda x : None if pd.isnull(x)
#                                     else None if str.lower(x) == 'nan'
#                                     else None if x.strip == ''
#                                     else x.strip())

In [16]:
stud.address.unique()

array(['U', nan, 'R'], dtype=object)

In [17]:
stud.sample(9)

,school,sex,age,address,famsize,pstatus,medu,fedu,mjob,fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
295,GP,M,17,U,GT3,T,3.0,3.0,health,other,home,mother,1.0,1.0,0.0,no,yes,yes,no,yes,yes,yes,no,4.0,4.0,3.0,5.0,4.0,55.0
266,GP,M,17,U,GT3,NaN,3.0,1.0,services,NaN,other,mother,1.0,NaN,0.0,no,no,yes,yes,yes,yes,yes,NaN,5.0,4.0,4.0,NaN,2.0,50.0
73,GP,M,16,U,GT3,T,3.0,1.0,other,other,NaN,mother,1.0,1.0,0.0,no,no,no,yes,yes,yes,no,no,NaN,3.0,2.0,5.0,2.0,70.0
374,MS,F,18,R,LE3,T,4.0,NaN,other,NaN,reputation,mother,2.0,3.0,0.0,no,no,no,no,yes,yes,yes,no,5.0,4.0,4.0,1.0,0.0,95.0
195,GP,F,17,U,NaN,T,2.0,NaN,services,services,course,father,1.0,2.0,0.0,no,NaN,no,yes,yes,yes,yes,yes,NaN,3.0,2.0,5.0,0.0,75.0
278,GP,F,18,U,GT3,T,4.0,4.0,health,health,reputation,father,1.0,2.0,1.0,yes,yes,no,yes,yes,yes,yes,yes,2.0,4.0,4.0,4.0,15.0,40.0
234,GP,M,16,U,LE3,NaN,1.0,1.0,other,other,home,mother,NaN,2.0,0.0,no,NaN,yes,no,yes,yes,yes,no,3.0,4.0,2.0,5.0,18.0,30.0
6,GP,M,16,NaN,LE3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,no,no,yes,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
135,GP,F,15,U,GT3,T,4.0,4.0,services,NaN,course,mother,1.0,3.0,0.0,no,yes,no,yes,yes,yes,yes,yes,4.0,3.0,3.0,5.0,0.0,0.0


### Распределение признака для числовых переменных

### Устранение выбросов

In [18]:
# и убедиться что ни чего не пропустил)
len(answers)

NameError: name 'answers' is not defined

### Оценка количества уникальных значений для номинативных переменных

### Преобразование данных

### Корреляционный анализ количественных элементов

### Отбор не коррелирующих переменных

### Анализ номинативных переменных, устранение не влияющих на предсказание конечной величины

In [ ]:
# на переменную scope

### Выводы о качестве данных

### Выводы о данных, которые подходят для построения дальнейшей модели

### Спасибо за внимание!